In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import json
import pandas as pd

In [ ]:
lines = []
with open("annotated/annotated.json") as fin:
    for line in fin.readlines():
        lines.append(json.loads(line))

lines = pd.DataFrame(lines)

lines['sentence_id'] = lines.meta.apply(lambda x: x['sentenceId'])

lines = pd.DataFrame(lines[['text', 'annotations', 'sentence_id']])

label_lookup = {
    21: "einfache Nennung",
    22: "Nennung mit Stereotyp",
    23: "Nennung mit negativen Gefühlen",
}

def parse_labels(label_list):
    parsed = []
    for label in label_list:
        parsed.append(
            label_lookup[label['label']]
        )
    return parsed

lines['annotations'] = lines.annotations.apply(parse_labels)
lines[lines.annotations.apply(len)>1]

In [ ]:
from lxml import etree

parser=etree.XMLParser(remove_comments=True)
tree = etree.fromstring(open("outdir/sentencized_charles-destinataires.xml").read(), parser=parser)

In [ ]:
def get_sentence_el(id_):
    result = tree.xpath(f"//tei:s[@id='{id_}']", namespaces={
        "tei":"http://www.tei-c.org/ns/1.0"
    })
    if len(result) == 1:
        return result[0]
    elif len(result) == 0:
        return None
        raise KeyError(f"id {id_} not found.")
    else:
        raise KeyError(f"multiple {id_} ids found.")

def get_type(el):
    type_ = None
    cel = el
    while type_ is None and cel.getparent() is not None:
        if 'type' in cel.attrib and cel.attrib['type'] in ["lettre", "journal", "note"]:
            type_ = cel.attrib['type']
        cel = cel.getparent()
    return type_

def get_chapter(el):
    chapter = None
    cel = el
    while chapter is None and cel.getparent() is not None:
        if 'type' in cel.attrib and cel.attrib['type'] == "chapter":
            chapter = cel.attrib['n']
        cel = cel.getparent()
    return chapter

In [ ]:
lines = pd.DataFrame(lines[lines.annotations.apply(len)>0])

lines['el'] = lines.sentence_id.apply(get_sentence_el)

lines = pd.DataFrame(lines[~lines['el'].isnull()])

lines['type'] = lines.el.apply(get_type)

lines['chapter'] = lines.el.apply(get_chapter)

In [ ]:
lines

In [ ]:
for (chapter, type_), subdf in lines.sort_values(["chapter", "type"]).groupby(["chapter", "type"]):
    print(chapter)
    print(type_)
    for _,row in subdf.iterrows():
        print(row.text)